# Notebook for Prediction - Mostly sklearn

## Data Filtering 

### Json to DataFrame

In [119]:
import os
import json
# !pip install pandas
# !pip install scikit-learn
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from pathlib import Path

def flatten_json(y, prefix=''):
    """Recursively flattens nested JSON"""
    out = {}
    for k, v in y.items():
        key = f"{prefix}{k}".replace(" ", "_").lower()
        if isinstance(v, dict):
            out.update(flatten_json(v, prefix=key + "_"))
        elif isinstance(v, list):
            out[key] = v #str(v)  # You can choose to serialize or extract specific features
        else:
            out[key] = v
    return out

def load_clients_data(base_path):
    client_rows = []
    clients_dir = Path(base_path) 
    sorted_clients = sorted(clients_dir.iterdir(), key=lambda x: int(x.name.split('_')[1]))

    for client_folder in tqdm(sorted_clients): #os.listdir(base_path):
        client_path = client_folder
        if not os.path.isdir(client_path):
            print(f"Warning: {client_path} is not a directory")
            continue

        client_data = {}
        for file_name in ['passport.json', 'client_profile.json', 'account_form.json', 'label.json']:
            file_path = os.path.join(client_path, file_name)
            if os.path.exists(file_path):
                with open(file_path, 'r', encoding='utf-8') as f:
                    try:
                        data = json.load(f)
                        flat_data = flatten_json(data)
                        client_data.update(flat_data)
                    except json.JSONDecodeError:
                        print(f"Warning: could not decode {file_path}")

        # Normalize label
        label = client_data.get('label', '').lower()
        client_data['label'] = 1 if label == 'accept' else 0

        client_rows.append(client_data)

    return pd.DataFrame(client_rows)

# Use the function
base_path = "datathon2025_2"  # Change this to your actual path
df = load_clients_data(base_path)


  9%|▉         | 905/10000 [00:00<00:01, 9041.02it/s]

100%|██████████| 10000/10000 [00:01<00:00, 7246.08it/s]


In [120]:
df

,first_name,middle_name,last_name,gender,country,country_code,nationality,birth_date,passport_number,passport_mrz,...,inheritance_details_inheritance_year,inheritance_details_profession,real_estate_details,investment_risk_profile,investment_horizon,investment_experience,type_of_mandate,preferred_markets,currency,label
0,Freja,Katrine,Christensen,F,Denmark,DNK,Danish,2002-04-18,UE2130779,[P<DNKCHRISTENSEN<<FREJA<KATRINE<<<<<<<<<<<<<<...,...,2020,Oil and Gas Executive,[],High,Medium,Inexperienced,Advisory,"[Spain, Denmark]",DKK,1
1,Thomas,Laurent,Louis,M,France,FRA,French,1990-02-07,OT9354543,[P<FRALEMAÎTRE<<THOMAS<LAURENT<<<<<<<<<<<<<<<<...,...,2016,Real Estate Developer,"[{'property type': 'townhouse', 'property valu...",Low,Medium,Experienced,Advisory,[France],EUR,0
2,Anna,Renate,vgger,F,Austria,AUT,Austrian,1985-06-22,CB5263486,[P<AUTEGGER<<ANNA<RENATE<<<<<<<<<<<<<<<<<<<<<<...,...,2017,Corporate Lawyer,"[{'property type': 'flat', 'property value': 9...",Moderate,Long-Term,Experienced,Advisory,[Austria],EUR,0
3,Virtanen,Laine,Viitanen,M,Finland,FIN,Finnish,1974-05-18,DX5798483,[P<FINVIITANEN<<VIRTANEN<LAINE<<<<<<<<<<<<<<<<...,...,2005,Investment Banker,"[{'property type': 'flat', 'property value': 7...",Low,Short,Experienced,Discretionary,"[Finland, Italy]",EUR,0
4,Martina,Sofia,Livi,F,Italy,ITA,Italian,1971-04-14,HO3030582,[P<ILIVI<<MARTINA<SOFIA<<<<<<<<<<<<<<<<<<<<<<<...,...,2006,Venture Capitalist,"[{'property type': 'house', 'property value': ...",Moderate,Medium,Expert,Advisory,"[Germany, Italy]",EUR,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Schipper,Wagemakers,De Jong,M,Netherlands,NLD,Dutch,1997-10-11,OQ9279820,[P<NLDDE JONG<<SCHIPPER<WAGEMAKERS<<<<<<<<<<<<...,...,2016,Private Equity Manager,[],Moderate,Short,Experienced,Discretionary,"[Belgium, Netherlands]",EUR,1
9996,Jasmijn,Daniëlle,Veenhoven,F,Netherlands,NLD,Dutch,1967-10-04,OW8331312,[P<NLDVEENHOVEN<<JASMIJN<DANIËLLE<<<<<<<<<<<<<...,...,2008,Private Equity Manager,"[{'property type': 'house', 'property value': ...",Considerable,Medium,Experienced,Advisory,[Netherlands],EUR,0
9997,Steiner,Maier,Haider,M,Austria,AUT,Austrian,1998-09-20,QW9934619,[P<AUTHAIDER<<STEINER<MAIER<<<<<<<<<<<<<<<<<<<...,...,2016,Real Estate Developer,[],High,Medium,Experienced,Advisory,[Austria],EUR,0
9998,Vogel,Meijer,Bakker,M,Netherlands,NLD,Dutch,1994-01-30,KA6647699,[P<NLDBAKKER<<VOGEL<MEIJER<<<<<<<<<<<<<<<<<<<<...,...,2018,Hedge Fund Manager,"[{'property type': 'villa', 'property value': ...",Moderate,Long-Term,Experienced,Advisory,"[Germany, Netherlands]",EUR,1


Filter out inconsistent clients since those are automatically rejected

In [121]:
import os
import csv 
import json

from tqdm import tqdm
from pathlib import Path
from extract_files import extract_files
from check_passport_consistency import passport_is_consistent
from check_account_form import account_form_is_consistent
from client_profile_graduation_year import profile_is_consistent
from cross_check_passport_client_profile_form import client_profile_and_passport_are_consistent
from cross_check_account_form_client_profile import account_form_and_client_profile_are_consistent
from cross_check_account_form_passport import account_form_and_passport_are_consistent

clients_dir = "datathon2025_2"  # Change this to your actual path

#NOTE: Probably not optimal, going over clients and sorting again but just keeping it simple for now
client_ids = []
predicted_labels = []
clients_dir = Path(clients_dir) 
sorted_clients = sorted(clients_dir.iterdir(), key=lambda x: int(x.name.split('_')[1]))
for client_dir in tqdm(sorted_clients):   
    client_ids.append(os.path.basename(client_dir))

    account_form_path = client_dir / "account_form.json"
    client_description_path = client_dir / "client_description.json"
    client_profile_path = client_dir / "client_profile.json"  
    passport_path = client_dir / "passport.json"

    account_form = json.load(account_form_path.open("r", encoding="utf-8"))
    client_description = json.load(client_description_path.open("r", encoding="utf-8"))
    client_profile = json.load(client_profile_path.open("r", encoding="utf-8"))
    passport = json.load(passport_path.open("r", encoding="utf-8"))

    if not passport_is_consistent(passport) or not account_form_is_consistent(account_form):
        predicted_labels.append(False)
        continue
    if not client_profile_and_passport_are_consistent(client_profile, passport):
        predicted_labels.append(False)
        continue
    if not account_form_and_client_profile_are_consistent(account_form, client_profile) or not account_form_and_passport_are_consistent(account_form, passport):
        predicted_labels.append(False)
        continue
    if not profile_is_consistent(client_profile):
        predicted_labels.append(False)
        continue
    predicted_labels.append(True)

  6%|▌         | 612/10000 [00:00<00:01, 6116.59it/s]

100%|██████████| 10000/10000 [00:01<00:00, 5897.23it/s]


In [122]:
# print(sum(predicted_labels) / len(predicted_labels))
filtered_df = df[predicted_labels].reset_index(drop=True)
filtered_df 

,first_name,middle_name,last_name,gender,country,country_code,nationality,birth_date,passport_number,passport_mrz,...,inheritance_details_inheritance_year,inheritance_details_profession,real_estate_details,investment_risk_profile,investment_horizon,investment_experience,type_of_mandate,preferred_markets,currency,label
0,Freja,Katrine,Christensen,F,Denmark,DNK,Danish,2002-04-18,UE2130779,[P<DNKCHRISTENSEN<<FREJA<KATRINE<<<<<<<<<<<<<<...,...,2020,Oil and Gas Executive,[],High,Medium,Inexperienced,Advisory,"[Spain, Denmark]",DKK,1
1,Schmitt,Scholz,Schwarz,M,Germany,DEU,German,1997-09-08,IL1661872,[P<DEUSCHWARZ<<SCHMITT<SCHOLZ<<<<<<<<<<<<<<<<<...,...,2015,Stock Trader,[],Considerable,Long-Term,Inexperienced,Advisory,[Germany],EUR,0
2,De Jong,Terpstra,Kuipers,M,Netherlands,NLD,Dutch,1997-11-02,PM4943954,[P<NLDKUIPERS<<DE JONG<TERPSTRA<<<<<<<<<<<<<<<...,...,2016,Investment Banker,[],Considerable,Long-Term,Experienced,Advisory,"[Italy, Netherlands]",EUR,1
3,Federica,Claudia,Malatesta,F,Italy,ITA,Italian,1966-01-27,RO6170391,[P<ITAMALATESTA<<FEDERICA<CLAUDIA<<<<<<<<<<<<<...,...,2013,Corporate Lawyer,"[{'property type': 'condo', 'property value': ...",Low,Medium,Experienced,Discretionary,[Italy],EUR,1
4,Fantini,Marino,Ferrari,M,Italy,ITA,Italian,1966-04-22,TZ2513112,[P<ITAFERRARI<<FANTINI< MARINO<<<<<<<<<<<<<<<<...,...,2012,Venture Capitalist,"[{'property type': 'house', 'property value': ...",Moderate,Medium,Expert,Advisory,"[Germany, Netherlands]",EUR,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6521,Evelyne,Fabienne,Roth,F,Switzerland,CHE,Swiss,1965-02-12,AL8697180,[P<CHEROTH<<EVELYNE<FABIENNE<<<<<<<<<<<<<<<<<<...,...,NaN,NaN,[],Low,Medium,Expert,Discretionary,[Switzerland],CHF,1
6522,Schipper,Wagemakers,De Jong,M,Netherlands,NLD,Dutch,1997-10-11,OQ9279820,[P<NLDDE JONG<<SCHIPPER<WAGEMAKERS<<<<<<<<<<<<...,...,2016,Private Equity Manager,[],Moderate,Short,Experienced,Discretionary,"[Belgium, Netherlands]",EUR,1
6523,Jasmijn,Daniëlle,Veenhoven,F,Netherlands,NLD,Dutch,1967-10-04,OW8331312,[P<NLDVEENHOVEN<<JASMIJN<DANIËLLE<<<<<<<<<<<<<...,...,2008,Private Equity Manager,"[{'property type': 'house', 'property value': ...",Considerable,Medium,Experienced,Advisory,[Netherlands],EUR,0
6524,Steiner,Maier,Haider,M,Austria,AUT,Austrian,1998-09-20,QW9934619,[P<AUTHAIDER<<STEINER<MAIER<<<<<<<<<<<<<<<<<<<...,...,2016,Real Estate Developer,[],High,Medium,Experienced,Advisory,[Austria],EUR,0


Removing Some Features

In [ ]:
# filtered_df 
# country_counts = filtered_df["country"].value_counts()
# print(country_counts)   #NOTE: Removing country as feature
# nationality_count = filtered_df["nationality"].value_counts()
marital_status_count = filtered_df["marital_status"].value_counts()   #NOTE: 3 values, will turn to categorical
print(marital_status_count)
# print(nationality_count) 
# inheritance_details_rs_count = filtered_df["inheritance_details_relationship"].value_counts()   #NOTE: 5 values, will turn to categorical
# print(inheritance_details_rs_count)
inheritance_details_profession_count = filtered_df["inheritance_details_profession"].value_counts()   #NOTE: 11 values, will turn to categorical
print(inheritance_details_profession_count)
filtered_df_ = filtered_df.drop(columns=["country","nationality","first_name","last_name","middle_name","passport_number","passport_issue_date","passport_expiry_date", "gender","country_code","passport_mrz","name","address_city","address_street_name","address_street_number","address_postal_code","email_address","secondary_school_name","country_of_domicile","phone_number"])
filtered_df__ = filtered_df_

current_date = pd.to_datetime("2025-04-01")
filtered_df__["birth_date"] = pd.to_datetime(filtered_df__["birth_date"], errors="coerce")
filtered_df__["inheritance_details_inheritance_year"] = pd.to_numeric(filtered_df__["inheritance_details_inheritance_year"], errors="coerce")
filtered_df__["secondary_school_graduation_year"] = pd.to_numeric(filtered_df__["secondary_school_graduation_year"], errors="coerce")

filtered_df__["age"] = filtered_df__["birth_date"].apply(lambda x: current_date.year - x.year - ((current_date.month, current_date.day) < (x.month, x.day)) if pd.notnull(x) else None)
filtered_df__["inheritance_age"] = filtered_df__["inheritance_details_inheritance_year"] - filtered_df__["birth_date"].dt.year# filtered_df__["secondary_school_graduation_age"] = filtered_df__["secondary_school_graduation_year"].apply(lambda x: current_date.year - x.year - ((current_date.month, current_date.day) < (x.month, x.day)) if pd.notnull(x) else None)
filtered_df__["secondary_school_graduation_age"] = filtered_df__["secondary_school_graduation_year"] - filtered_df__["birth_date"].dt.year
filtered_df__["number_of_universities"] = filtered_df__["higher_education"].apply(lambda x: len(x) if isinstance(x, list) else 0)

filtered_df__["earliest_university_graduation_age"] = filtered_df__.apply(
    lambda row: min([int(university["graduation_year"]) for university in row["higher_education"]]) - row["birth_date"].year
    if isinstance(row["higher_education"], list) and len(row["higher_education"]) > 0 else None,
    axis=1
)
filtered_df__["latest_university_graduation_age"] = filtered_df__.apply(
    lambda row: max([int(university["graduation_year"]) for university in row["higher_education"]]) - row["birth_date"].year
    if isinstance(row["higher_education"], list) and len(row["higher_education"]) > 0 else None,
    axis=1
)

current_year = 2025  # Assuming current year is 2025
filtered_df__["total_years_of_employment"] = filtered_df__["employment_history"].apply(
    lambda x: sum([(current_year - job["start_year"]) if job["end_year"] is None else (job["end_year"] - job["start_year"]) for job in x])
    if isinstance(x, list) else 0
)
filtered_df__["num_jobs"] = filtered_df__["employment_history"].apply(
    lambda x: len(x) if isinstance(x, list) else 0
)
filtered_df__["longest_job_duration"] = filtered_df__["employment_history"].apply(
    lambda x: max([(current_year - job["start_year"]) if job["end_year"] is None else (job["end_year"] - job["start_year"]) for job in x], default=0)
    if isinstance(x, list) else 0
)
filtered_df__["average_salary"] = filtered_df__["employment_history"].apply(
    lambda x: sum([job["salary"] for job in x if job["salary"] is not None]) / len(x) if isinstance(x, list) and len(x) > 0 else None
)
filtered_df__["most_recent_job_end_age"] = filtered_df__["employment_history"].apply(
    lambda x: max([job["end_year"] - filtered_df__["birth_date"].dt.year[i] if job["end_year"] is not None 
                   else current_year - filtered_df__["birth_date"].dt.year[i] for i, job in enumerate(x)], default=None)
    if isinstance(x, list) else None
)
filtered_df__["most_recent_job_start_age"] = filtered_df__["employment_history"].apply(
    lambda x: max([job["start_year"] - filtered_df__["birth_date"].dt.year[i] for i, job in enumerate(x)], default=None) 
    if isinstance(x, list) else None
)

# print(filtered_df__["employment_history"][5])

# filtered_df__["higher_education"][5]
# filtered_df__["real_estate_details"][5]
property_types = filtered_df__["real_estate_details"].apply(
    lambda x: [property['property type'] for property in x] if isinstance(x, list) else []
)

# Flatten the list of lists and get unique values
unique_property_types = set([item for sublist in property_types for item in sublist])       #NOTE: This was to get the available property types

# Print all unique property types
# print(unique_property_types)

property_types = ['flat', 'villa', 'townhouse', 'condo', 'house']

filtered_df__["total_property_count"] = filtered_df__["real_estate_details"].apply(
    lambda x: len(x) if isinstance(x, list) else 0
)

# Calculate total investment value across all properties
filtered_df__["total_property_value"] = filtered_df__["real_estate_details"].apply(
    lambda x: sum(property.get('property value', 0) for property in x) if isinstance(x, list) else 0
)

# Count distinct property types (e.g., flat, villa, condo)
filtered_df__["num_property_types"] = filtered_df__["real_estate_details"].apply(
    lambda x: len(set(property.get('property type') for property in x)) if isinstance(x, list) else 0
)

# Calculate max property value
filtered_df__["max_property_value"] = filtered_df__["real_estate_details"].apply(
    lambda x: max((property.get('property value', 0) for property in x), default=0) if isinstance(x, list) else 0
)

# Calculate min property value
filtered_df__["min_property_value"] = filtered_df__["real_estate_details"].apply(
    lambda x: min((property.get('property value', 0) for property in x), default=0) if isinstance(x, list) else 0
)

filtered_df__ = pd.get_dummies(filtered_df__, columns=["marital_status"], drop_first=False)
filtered_df__ = pd.get_dummies(filtered_df__, columns=["inheritance_details_profession"], drop_first=False)
filtered_df__ = pd.get_dummies(filtered_df__, columns=["inheritance_details_relationship"], drop_first=False)
filtered_df__ = pd.get_dummies(filtered_df__, columns=["investment_risk_profile"], drop_first=False)
filtered_df__ = pd.get_dummies(filtered_df__, columns=["investment_horizon"], drop_first=False)
filtered_df__ = pd.get_dummies(filtered_df__, columns=["investment_experience"], drop_first=False)
filtered_df__ = pd.get_dummies(filtered_df__, columns=["type_of_mandate"], drop_first=False)
filtered_df__ = pd.get_dummies(filtered_df__, columns=["currency"], drop_first=False)

available_markets = ['Spain', 'Denmark', 'Germany', 'Italy', 'Netherlands', 'France', 'Finland', 'Switzerland', 'Belgium', 'Austria']

# Convert 'preferred_markets' into categorical features (one per available market)
for market in available_markets:
    filtered_df__[market] = df["preferred_markets"].apply(lambda x: 1 if market in x else 0)

filtered_df__.drop(columns=["birth_date","inheritance_details_inheritance_year","secondary_school_graduation_year",
"higher_education","employment_history","real_estate_details", "preferred_markets"], inplace=True)


filtered_df__ = filtered_df__.apply(lambda x: x.astype(int) if x.dtype == 'bool' else x)

# all_markets = filtered_df__["preferred_markets"].explode().dropna().unique()

# # Print the unique markets
# print(all_markets)
# pd.set_option('display.max_columns', None)

filtered_df__

marital_status
married     3083
single      1732
divorced    1061
widowed      650
Name: count, dtype: int64
inheritance_details_profession
Investment Banker         524
Hedge Fund Manager        513
Real Estate Developer     510
Neurosurgeon              509
Oil and Gas Executive     503
Private Equity Manager    500
Corporate Lawyer          500
Venture Capitalist        493
Tech Entrepreneur         491
Stock Trader              465
                           41
Name: count, dtype: int64


,aum_savings,aum_inheritance,aum_real_estate_value,label,age,inheritance_age,secondary_school_graduation_age,number_of_universities,earliest_university_graduation_age,latest_university_graduation_age,total_years_of_employment,num_jobs,longest_job_duration,average_salary,most_recent_job_end_age,most_recent_job_start_age,total_property_count,total_property_value,num_property_types,max_property_value,min_property_value,marital_status_divorced,marital_status_married,marital_status_single,marital_status_widowed,inheritance_details_profession_,inheritance_details_profession_Corporate Lawyer,inheritance_details_profession_Hedge Fund Manager,inheritance_details_profession_Investment Banker,inheritance_details_profession_Neurosurgeon,inheritance_details_profession_Oil and Gas Executive,inheritance_details_profession_Private Equity Manager,inheritance_details_profession_Real Estate Developer,inheritance_details_profession_Stock Trader,inheritance_details_profession_Tech Entrepreneur,inheritance_details_profession_Venture Capitalist,inheritance_details_relationship_,inheritance_details_relationship_father,inheritance_details_relationship_grandfather,inheritance_details_relationship_grandmother,inheritance_details_relationship_mother,investment_risk_profile_,investment_risk_profile_Aggressive,investment_risk_profile_Balanced,investment_risk_profile_Conservative,investment_risk_profile_Considerable,investment_risk_profile_High,investment_risk_profile_Low,investment_risk_profile_Moderate,investment_horizon_1-10 months,investment_horizon_1-2 months,investment_horizon_1-3 months,investment_horizon_1-4 months,investment_horizon_1-5 months,investment_horizon_1-6 months,investment_horizon_1-7 months,investment_horizon_1-8 months,investment_horizon_1-9 months,investment_horizon_10 months,investment_horizon_10 weeks,investment_horizon_2 months,investment_horizon_2 weeks,investment_horizon_3 months,investment_horizon_3 weeks,investment_horizon_4 months,investment_horizon_4 weeks,investment_horizon_5 months,investment_horizon_5 weeks,investment_horizon_6 months,investment_horizon_6 weeks,investment_horizon_7 months,investment_horizon_7 weeks,investment_horizon_8 months,investment_horizon_8 weeks,investment_horizon_9 months,investment_horizon_9 weeks,investment_horizon_Long-Term,investment_horizon_Medium,investment_horizon_Short,investment_experience_Experienced,investment_experience_Expert,investment_experience_Inexperienced,type_of_mandate_,type_of_mandate_Advisory,type_of_mandate_Discretionary,type_of_mandate_Execution-Only,type_of_mandate_Hybrid,currency_CHF,currency_DKK,currency_EUR,Spain,Denmark,Germany,Italy,Netherlands,France,Finland,Switzerland,Belgium,Austria
0,0,13140000,0,1,22,18.0,20,0,NaN,NaN,0,0,0,NaN,NaN,NaN,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0
1,20000,2420000,0,0,27,18.0,19,1,24.0,24.0,3,1,3,45000.0,23.0,20.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
2,20000,1350000,0,1,27,19.0,20,1,24.0,24.0,3,1,3,43000.0,23.0,20.0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
3,1450000,2690000,2825000,1,59,47.0,17,1,22.0,22.0,29,5,19,215000.0,59.0,40.0,2,2825000,2,1830000,995000,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0
4,1590000,670000,1790000,0,58,46.0,20,1,23.0,23.0,31,5,26,231600.0,59.0,33.0,1,1790000,1,1790000,1790000,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

Exporting X and y to csv files

In [146]:
X = filtered_df__.drop(columns=["label"])  # Drop the 'label' column for features
y = filtered_df__["label"]  # The 'label' column will be the target variable

# Save X and y into separate CSV files
X.to_csv("X.csv", index=True)  # Save features to 'features.csv'
y.to_csv("y.csv", index=True)    # Save labels to 'labels.csv'